In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Load  Data

In [7]:
mnist_full = pd.read_csv('./data/train.csv')
mnist_test = pd.read_csv('./data/test.csv')
mnist_full

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
mnist_full['label'].value_counts()

1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64

In [12]:
y = mnist_full['label'].to_numpy()
y

array([1, 0, 1, ..., 7, 6, 9])

In [13]:
X = mnist_full.drop(columns='label').to_numpy().reshape(-1, 28, 28, 1)
X.shape

(42000, 28, 28, 1)

### Split Data

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

In [16]:
X_train.shape

(31500, 28, 28, 1)

## Build Model

In [17]:
import tensorflow as tf
from tensorflow.keras import *

In [18]:
model = Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(64, (4,4), activation='relu'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

Metal device set to: Apple M1


2022-03-03 19:27:58.259166: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-03 19:27:58.259304: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 64)        1088      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 12, 12, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 128)        0         
 2D)                                                             
                                                        

In [20]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [21]:
model.fit(X_train, y_train, 
          epochs=50, 
          batch_size=100, 
          validation_data=[X_val, y_val], 
          validation_batch_size=100, 
          verbose=1)

2022-03-03 19:27:58.727000: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-03 19:27:58.875312: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/50
315/315 [==============================] - ETA: 0s - loss: 0.1538 - acc: 0.9534

2022-03-03 19:28:05.775919: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


315/315 [==============================] - 8s 21ms/step - loss: 0.1538 - acc: 0.9534 - val_loss: 0.0743 - val_acc: 0.9806
Epoch 2/50
315/315 [==============================] - 7s 21ms/step - loss: 0.0570 - acc: 0.9828 - val_loss: 0.0816 - val_acc: 0.9802
Epoch 3/50
315/315 [==============================] - 6s 21ms/step - loss: 0.0401 - acc: 0.9883 - val_loss: 0.0523 - val_acc: 0.9860
Epoch 4/50
315/315 [==============================] - 6s 20ms/step - loss: 0.0303 - acc: 0.9907 - val_loss: 0.0667 - val_acc: 0.9840
Epoch 5/50
315/315 [==============================] - 6s 21ms/step - loss: 0.0241 - acc: 0.9925 - val_loss: 0.0816 - val_acc: 0.9823
Epoch 6/50
315/315 [==============================] - 7s 21ms/step - loss: 0.0254 - acc: 0.9919 - val_loss: 0.0673 - val_acc: 0.9865
Epoch 7/50
315/315 [==============================] - 6s 21ms/step - loss: 0.0219 - acc: 0.9935 - val_loss: 0.0515 - val_acc: 0.9891
Epoch 8/50
315/315 [==============================] - 6s 21ms/step - loss: 0.013

## Retrain on entire data

In [22]:
model.fit(X, y, epochs=5, batch_size=100)

Epoch 1/5
420/420 [==============================] - 8s 19ms/step - loss: 0.0223 - acc: 0.9958
Epoch 2/5
420/420 [==============================] - 8s 19ms/step - loss: 0.0102 - acc: 0.9975
Epoch 3/5
420/420 [==============================] - 8s 20ms/step - loss: 0.0077 - acc: 0.9980
Epoch 4/5
420/420 [==============================] - 8s 20ms/step - loss: 0.0041 - acc: 0.9987
Epoch 5/5
420/420 [==============================] - 8s 20ms/step - loss: 0.0038 - acc: 0.9988


### Predict on Test data

In [23]:
mnist_test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
X_test = mnist_test.to_numpy().reshape(-1, 28, 28, 1)
X_test.shape

(28000, 28, 28, 1)

In [26]:
y_pred = model.predict(X_test)
y_pred

array([[3.9395519e-35, 7.1668879e-29, 1.0000000e+00, ..., 8.9069884e-29,
        3.4538183e-21, 2.6241214e-31],
       [1.0000000e+00, 2.0330358e-25, 3.2150569e-19, ..., 2.5782829e-20,
        6.2092114e-17, 1.7206665e-14],
       [1.5534507e-20, 1.7508007e-16, 2.5474086e-14, ..., 9.6418672e-11,
        2.3758510e-12, 1.0000000e+00],
       ...,
       [0.0000000e+00, 3.0859565e-37, 5.2700548e-34, ..., 8.7242387e-38,
        3.4238781e-35, 0.0000000e+00],
       [1.0221193e-14, 7.2403681e-17, 1.8384152e-13, ..., 4.8022247e-10,
        1.7228288e-12, 1.0000000e+00],
       [0.0000000e+00, 1.2906413e-33, 1.0000000e+00, ..., 2.6109164e-35,
        1.9511524e-24, 0.0000000e+00]], dtype=float32)

In [29]:
predictions = y_pred.argmax(axis=-1)
predictions

array([2, 0, 9, ..., 3, 9, 2])

## Create Submission

In [30]:
submission = pd.DataFrame({'ImageId': range(1, len(y_pred) + 1), 'Label': predictions})
submission

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [31]:
submission.to_csv('data/submission.csv', index=False)

resnet_v2 = applications.ResNet50V2(include_top=False)

resnet_v2.summary()

for layer in resnet_v2.layers:
    layer.trainable = False

resnet_v2 = applications.ResNet50V2(include_top=False)

resnet_v2.summary()

for layer in resnet_v2.layers:
    layer.trainable = False

resnet_v2 = applications.ResNet50V2(include_top=False)

resnet_v2.summary()

for layer in resnet_v2.layers:
    layer.trainable = False

resnet_v2 = applications.ResNet50V2(include_top=False)

resnet_v2.summary()

for layer in resnet_v2.layers:
    layer.trainable = False